# Reproduce of the paper *Spectral Networks and Deep Locally Connected Networks on Graphs*

Here, Spectral Network and Deep Locally Connected Network is tested on the CORA dataset.

In [1]:
!pip install torch_geometric
!pip install networkx
!pip install python-louvain
!pip install pymetis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
# Networks
import torch, numpy
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj
import community.community_louvain as community_louvain
import networkx as nx
import pymetis

class SpectralConvolution(nn.Module):
    def __init__(self, in_channels,  out_channels, edge_index, first_k = 64):
        super(SpectralConvolution, self).__init__()
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.edge_index = edge_index
        self.first_k = first_k
        self.adj_matrix = to_dense_adj(edge_index)[0]
        self.laplacian = self.compute_laplacian(self.adj_matrix)
        self.eigvals, self.eigvecs = self.compute_eigenvectors(self.laplacian)
        # print(self.eigvals)
        self.selected_eigvecs = self.eigvecs[:, -first_k:].to('cuda')
        
        self.filters = nn.Parameter(torch.randn(out_channels, first_k, in_channels))
        # print(self.filters.shape)
        print("#parameters",out_channels*first_k*in_channels)
    
    def forward(self, x):
        x = torch.matmul(self.selected_eigvecs.T, x)
        xs = [
            (flt * x).sum( dim = -1, keepdim = True)
            for flt in self.filters
        ]
        x = torch.cat(xs,dim = -1)
        x = torch.matmul(self.selected_eigvecs, x)
        return x
    
    def compute_laplacian(self, adj_matrix):
        degree = torch.sum(adj_matrix, dim=1)
        laplacian = torch.diag(degree) - adj_matrix
        return laplacian
    
    def compute_eigenvectors(self, laplacian):
        eigvals, eigvecs = torch.symeig(laplacian, eigenvectors=True)
        return eigvals, eigvecs

class SpectralNetwork(nn.Module):
    def __init__(self, edge_index, in_channels, hidden_channels, out_channels, num_class):
        super(SpectralNetwork, self).__init__()
        self.conv1 = SpectralConvolution(
            in_channels = hidden_channels,
            out_channels = hidden_channels, 
            edge_index = edge_index, 
        )
        self.conv2 = SpectralConvolution(
            in_channels = hidden_channels,
            out_channels = out_channels, 
            edge_index = edge_index, 
        )
        self.linear = nn.Linear(out_channels, num_class)
        print("#parameters",out_channels * num_class)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.linear(x)
        return x

class Pooling(nn.Module):
    def __init__(self, clustering):
        '''
            clustering: # clusters x # nodes, i-th row indicates the nodes in the i-th cluster
        '''
        super(Pooling, self).__init__()
        clustering = clustering / clustering.sum(dim = 1, keepdim = True).expand_as(clustering)
        self.clustering = clustering.to('cuda')

    def forward(self, x):
        x = self.clustering @ x
        return x

def edge_index_to_adj_list(edge_index):
    adj_list = [[] for _ in range(torch.max(edge_index) + 1)]
    for i in range(edge_index.shape[1]):
        adj_list[edge_index[0][i]].append(edge_index[1][i])
    return adj_list

def metis(edge_index, nc = 500):
    adjacency_list = edge_index_to_adj_list(edge_index)
    n_cuts, membership = pymetis.part_graph(nc, adjacency=adjacency_list)
    membership = torch.tensor(membership)
    clustering = torch.zeros(nc, len(membership))
    for i in range(nc):
        clustering[i][membership == i] = 1
    return nc, clustering

def louvain(G):
    partition = community_louvain.best_partition(G)
    nc_louvain = len(numpy.unique( [partition[nodes] for nodes in partition.keys()] ))
    n = len(G.nodes())

    clusters = []
    k = 0
    for com in set(partition.values()):
        list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
        cluster = torch.zeros(n)
        cluster[list_nodes] = 1.0
        k += 1
        clusters.append(cluster)
    clustering = torch.stack(clusters, dim = 0)
    return nc_louvain, clustering

def clustering(edge_index, nc = 500):
    '''
        Input edge index matrix, output new edge_index matrix and clustering map
    '''
    G = nx.from_edgelist(edge_index.T.numpy())
    n = len(G.nodes())

    # nc, clustering = metis(edge_index, nc)
    nc, clustering = louvain(G)

    new_edge_index = []
    A = nx.adjacency_matrix(G).todense()
    for start in range(nc):
        for end in range(nc):
            if start == end:
                continue
            rows = A[clustering[start].bool()]
            rows_columns = rows[:,clustering[end].bool()]
            if rows_columns.sum()>0:
                new_edge_index.append([start, end])

    new_edge_index = torch.tensor(new_edge_index).T
    return clustering, new_edge_index

class LocalConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, edge_index):
        super(LocalConvolution, self).__init__()
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.edge_index = edge_index
        self.adj_matrix = to_dense_adj(edge_index)[0]
        self.mask = (self.adj_matrix > 0).float()
        filters = [nn.ParameterList(
                [nn.Parameter(
                    (torch.randn(*self.adj_matrix.shape) * self.mask).to_sparse()) 
                  for i in range(in_channels)
                ]) for o in range(out_channels) ]
        
        self.filters = nn.ParameterList(filters)
        # self.mask = self.mask.to("cuda")
        print("#parameters",(self.mask.sum() * self.out_channels * self.in_channels).item())
    
    def forward(self, x):
        xs = []
        for o in range(self.out_channels):
          temp = 0
          for i in range(self.in_channels):
            flt = self.filters[o][i]
            temp = temp + torch.sparse.mm(flt, x[:,i:i+1])
          xs.append(temp)
        x = torch.cat(xs,dim = -1)
        return x

class NodeDeepLocallyConnectedNetwork(nn.Module):
    def __init__(self, edge_index, in_channels, hidden_channels, out_channels, num_class):
        super(NodeDeepLocallyConnectedNetwork, self).__init__()
        self.ds = nn.Linear(in_channels, hidden_channels)
        print("#parameters",in_channels * hidden_channels)
        self.adj_matrix = to_dense_adj(edge_index)[0]
        self.num_node = self.adj_matrix.shape[0]
        clustering1, edge_index1 = clustering(edge_index)
        self.pooling1 = Pooling(clustering1)
        self.conv1 = LocalConvolution(
            in_channels = hidden_channels,
            out_channels = hidden_channels, 
            edge_index = edge_index1, 
        )
        self.conv2 = LocalConvolution(
            in_channels = hidden_channels,
            out_channels = out_channels, 
            edge_index = edge_index1, 
        )
        self.linear = nn.Linear(out_channels, num_class)
        self.clustering1 = clustering1
        print("#parameters",out_channels * num_class)
    
    def forward(self, x):
        x = F.relu(self.ds(x))
        x = self.pooling1(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.linear(x)
        out = torch.zeros(self.num_node, x.shape[1]).to(device = x.device)
        for r,indices in enumerate(self.clustering1):
          out[indices.bool()] += x[r:r+1].expand_as(out[indices.bool()]) 
        return out

class FFN(nn.Module):
    def __init__(self, hidden_channels, out_channels, num_class, in_channels = 1433):
        super(FFN, self).__init__()
        self.fc1 = nn.Linear(in_channels, hidden_channels)
        print("#parameters",in_channels * hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, out_channels)
        print("#parameters",hidden_channels * out_channels)
        self.linear = nn.Linear(out_channels, num_class)
        print("#parameters",out_channels * num_class)

        # he initialization
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_in')

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.linear(x)


In [3]:
# Training/Testing
from torch_geometric.datasets import Planetoid
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam, SGD

cora = Planetoid(root='./data', name='Cora')[0]
print(cora)
x, y = cora.x.to('cuda'), cora.y.to('cuda')

def train(model, epochs):
  print("Train:")
  opt = SGD(model.parameters(), 0.01, weight_decay=5e-4)
  model.train()
  for i in range(epochs):
    logit = model(x)
    loss = F.cross_entropy(logit[cora.train_mask], y[cora.train_mask])
    loss.backward()
    opt.step()
    opt.zero_grad()
    print("epoch",i,"loss",loss.item())
  return model

def test(model):
  model.eval()
  logit = model(x)
  right_n = torch.argmax(logit[cora.test_mask], 1) == y[cora.test_mask]
  acc = right_n.sum()/cora.test_mask.sum() * 100
  print("Test Acc: ", acc.item(), "%")

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [ ]:
sn = SpectralNetwork(
      edge_index = cora.edge_index,
      in_channels = 1433, 
      hidden_channels = 512, 
      out_channels = 128, 
      num_class = 7).to('cuda')
sn = train(sn, 100)
test(sn)

#parameters 46956544
#parameters 4194304
#parameters 896
Train:
epoch 0 loss 3.106330633163452
epoch 1 loss 1.8261003494262695
epoch 2 loss 1.6371712684631348
epoch 3 loss 1.5152947902679443
epoch 4 loss 1.4363800287246704
epoch 5 loss 1.3796617984771729
epoch 6 loss 1.3391594886779785
epoch 7 loss 1.309530258178711
epoch 8 loss 1.2817682027816772
epoch 9 loss 1.2558258771896362
epoch 10 loss 1.2342019081115723
epoch 11 loss 1.2163019180297852
epoch 12 loss 1.2011054754257202
epoch 13 loss 1.1899466514587402
epoch 14 loss 1.1761223077774048
epoch 15 loss 1.165631651878357
epoch 16 loss 1.156022548675537
epoch 17 loss 1.147183895111084
epoch 18 loss 1.139008641242981
epoch 19 loss 1.1314969062805176
epoch 20 loss 1.1245543956756592
epoch 21 loss 1.1181000471115112
epoch 22 loss 1.1121307611465454
epoch 23 loss 1.1065647602081299
epoch 24 loss 1.1013716459274292
epoch 25 loss 1.0965321063995361
epoch 26 loss 1.0920215845108032
epoch 27 loss 1.0877217054367065
epoch 28 loss 1.083618044853

In [18]:
dlcn = NodeDeepLocallyConnectedNetwork(
      edge_index = cora.edge_index, 
      in_channels = 1433,
      hidden_channels = 32, 
      out_channels = 16, 
      num_class = 7).to('cuda')
dlcn = train(dlcn, 100)
test(dlcn)

#parameters 45856
#parameters 1531904.0
#parameters 765952.0
#parameters 112
Train:
epoch 0 loss 6.896874904632568
epoch 1 loss 10.542612075805664
epoch 2 loss 17.057861328125
epoch 3 loss 3.0981805324554443
epoch 4 loss 2.385615825653076
epoch 5 loss 1.754275918006897
epoch 6 loss 1.1865519285202026
epoch 7 loss 1.0509910583496094
epoch 8 loss 0.9839080572128296
epoch 9 loss 0.9468902349472046
epoch 10 loss 0.9235628247261047
epoch 11 loss 0.8991420269012451
epoch 12 loss 0.8812931180000305
epoch 13 loss 0.8670950531959534
epoch 14 loss 0.8397684097290039
epoch 15 loss 0.819165825843811
epoch 16 loss 0.8006839156150818
epoch 17 loss 0.7950226664543152
epoch 18 loss 0.8098656535148621
epoch 19 loss 0.8424105048179626
epoch 20 loss 0.8015243411064148
epoch 21 loss 0.75485759973526
epoch 22 loss 0.7362593412399292
epoch 23 loss 0.7230392694473267
epoch 24 loss 0.7135909199714661
epoch 25 loss 0.7056167721748352
epoch 26 loss 0.6983850598335266
epoch 27 loss 0.6926122903823853
epoch 28 lo

In [39]:
ffn = FFN(
      hidden_channels = 1024, 
      out_channels = 512, 
      num_class = 7).to('cuda')
ffn = train(ffn, 100)
test(ffn)

#parameters 1467392
#parameters 524288
#parameters 3584
Train:
epoch 0 loss 1.9660688638687134
epoch 1 loss 1.963498592376709
epoch 2 loss 1.9609383344650269
epoch 3 loss 1.9583853483200073
epoch 4 loss 1.9558415412902832
epoch 5 loss 1.953304648399353
epoch 6 loss 1.9507803916931152
epoch 7 loss 1.948268175125122
epoch 8 loss 1.9457645416259766
epoch 9 loss 1.94326913356781
epoch 10 loss 1.9407843351364136
epoch 11 loss 1.9383095502853394
epoch 12 loss 1.9358415603637695
epoch 13 loss 1.9333783388137817
epoch 14 loss 1.9309210777282715
epoch 15 loss 1.9284683465957642
epoch 16 loss 1.9260201454162598
epoch 17 loss 1.923575758934021
epoch 18 loss 1.9211384057998657
epoch 19 loss 1.9187090396881104
epoch 20 loss 1.9162852764129639
epoch 21 loss 1.9138659238815308
epoch 22 loss 1.9114519357681274
epoch 23 loss 1.9090442657470703
epoch 24 loss 1.9066404104232788
epoch 25 loss 1.9042432308197021
epoch 26 loss 1.9018528461456299
epoch 27 loss 1.899467945098877
epoch 28 loss 1.89708650112152